### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use Jupyter gmaps and the Google Places API for this part of the assignment.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

As final considerations:

* Create a new GitHub repository for this project called `API-Challenge` (note the kebab-case). **Do not add to an existing repo**
* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part I You must include a written description of three observable trends based on the data.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* See [Example Solution](WeatherPy_Example.pdf) for a reference on expected format for Part I.
* For max intensity in the heat map try setting it to the highest humidity found in the data set.

## Hints and Considerations

* The city data is generated based on random coordinates as well as different times of the year; as such, your outputs will not be an exact match to the provided starter notebook.

* You may want to start this assignment by refreshing yourself on the [geographic coordinate system](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm).

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer  basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

* A starter code for Citipy has been provided. However, if you're craving an extra challenge, push yourself to learn how it works: [citipy Python library](https://pypi.python.org/pypi/citipy). Before you try to incorporate the library into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

* Part of our expectation in this challenge is that you will use critical thinking skills to understand how and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

* In building your script, pay attention to the cities you are using in your query pool. Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 500 cities concentrated in one region of the world? Even if you were a geographic genius, simply rattling 500 cities based on your human selection would create a biased dataset. Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

* Once you have computed the linear regression for one chart, the process will be similar for all others. As a bonus, try to create a function that will create these charts based on different parameters.

* Remember that each coordinate will trigger a separate call to the Google API. If you're creating your own criteria to plan your vacation, try to reduce the results in your DataFrame to 10 or fewer cities.

* Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

### Copyright

Trilogy Education Services © 2019. All Rights Reserved.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#load csv into dataframe
load_csv = "output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hobart,100,AU,1569513989,71,-42.88,147.33,53.01,10.29
1,Hay River,75,CA,1569513990,66,60.82,-115.79,32.00,11.21
2,Rikitea,41,PF,1569513992,73,-23.12,-134.97,71.23,10.18
3,Carutapera,86,BR,1569513993,64,-1.20,-46.02,85.99,9.04
4,Soyo,40,AO,1569513994,78,-6.13,12.37,80.60,10.29


In [3]:
#humidity heatmap

locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,27,Carnarvon,0,ZA,1569514022,8,-30.97,22.13,76.45,8.50
1,99,Kruisfontein,0,ZA,1569514118,28,-34.00,24.73,79.87,4.21
2,131,San Patricio,0,PY,1569514160,35,-26.98,-56.83,78.43,5.46
3,152,Hami,0,CN,1569514188,16,42.84,93.51,74.11,5.06
4,153,Tautira,0,PF,1569514189,78,-17.73,-149.15,75.20,3.36
5,154,Tezu,0,IN,1569514191,83,27.93,96.16,70.33,3.98
6,167,Guane,0,CU,1569514208,72,22.20,-84.09,79.69,8.32
7,181,Saldanha,0,PT,1569514226,27,41.42,-6.55,75.91,2.10
8,224,Shenjiamen,0,CN,1569514287,73,29.96,122.30,71.77,6.80
9,236,Garlasco,0,IT,1569514304,60,45.20,8.92,77.00,3.36


In [112]:
# create hotel_df with hotel name column
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Carnarvon,0,ZA,1569514022,8,-30.97,22.13,76.45,8.50,NaN
1,Kruisfontein,0,ZA,1569514118,28,-34.00,24.73,79.87,4.21,NaN
2,San Patricio,0,PY,1569514160,35,-26.98,-56.83,78.43,5.46,NaN
3,Hami,0,CN,1569514188,16,42.84,93.51,74.11,5.06,NaN
4,Tautira,0,PF,1569514189,78,-17.73,-149.15,75.20,3.36,NaN
5,Tezu,0,IN,1569514191,83,27.93,96.16,70.33,3.98,NaN
6,Guane,0,CU,1569514208,72,22.20,-84.09,79.69,8.32,NaN
7,Saldanha,0,PT,1569514226,27,41.42,-6.55,75.91,2.10,NaN
8,Shenjiamen,0,CN,1569514287,73,29.96,122.30,71.77,6.80,NaN
9,Garlasco,0,IT,1569514304,60,45.20,8.92,77.00,3.36,NaN


In [99]:

# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.

#search googleplaces for hotels within 5000 meters

# geocoordinates
# target_coordinates = hotel_df[["Lat", "Lng"]]
resultss = []
target_coordinates = '-30.97 , 22.13'
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
hotel_results = response.json()
resultss.append(hotel_results['results'][0]['name'])

resultss


['Tip Top Guesthouses']

In [108]:
# hotel_df['Lat'][0]
x=0
temp = []
for cities in hotel_df['City']:
    temp.append(str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x]))
    x+=1
temp


['-30.97 , 22.13',
 '-34.0 , 24.73',
 '-26.98 , -56.83',
 '42.84 , 93.51',
 '-17.73 , -149.15',
 '27.93 , 96.16',
 '22.2 , -84.09',
 '41.42 , -6.55',
 '29.96 , 122.3',
 '45.2 , 8.92',
 '43.6 , 73.76',
 '13.8 , 121.11',
 '-28.66 , -56.0',
 '35.24 , 60.63',
 '-36.78 , -59.86',
 '-22.46 , -68.93',
 '-34.02 , 20.44',
 '39.12 , 66.88',
 '26.88 , -111.98',
 '38.44 , 57.43']

In [113]:
hotels = []
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a     
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

hotel_df['Hotel Name'] = hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Carnarvon,0,ZA,1569514022,8,-30.97,22.13,76.45,8.50,NaN
1,Kruisfontein,0,ZA,1569514118,28,-34.00,24.73,79.87,4.21,NaN
2,San Patricio,0,PY,1569514160,35,-26.98,-56.83,78.43,5.46,NaN
3,Hami,0,CN,1569514188,16,42.84,93.51,74.11,5.06,NaN
4,Tautira,0,PF,1569514189,78,-17.73,-149.15,75.20,3.36,NaN
5,Tezu,0,IN,1569514191,83,27.93,96.16,70.33,3.98,NaN
6,Guane,0,CU,1569514208,72,22.20,-84.09,79.69,8.32,NaN
7,Saldanha,0,PT,1569514226,27,41.42,-6.55,75.91,2.10,NaN
8,Shenjiamen,0,CN,1569514287,73,29.96,122.30,71.77,6.80,NaN
9,Garlasco,0,IT,1569514304,60,45.20,8.92,77.00,3.36,NaN


In [110]:
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Carnarvon,0,ZA,1569514022,8,-30.97,22.13,76.45,8.50,Tip Top Guesthouses
1,Kruisfontein,0,ZA,1569514118,28,-34.00,24.73,79.87,4.21,Oyster Bay House Rental
2,San Patricio,0,PY,1569514160,35,-26.98,-56.83,78.43,5.46,Posada Doña Antonia
3,Hami,0,CN,1569514188,16,42.84,93.51,74.11,5.06,Post And Telecommunication Hotel
4,Tautira,0,PF,1569514189,78,-17.73,-149.15,75.20,3.36,Pension OAOA
5,Tezu,0,IN,1569514191,83,27.93,96.16,70.33,3.98,Circuit House
6,Guane,0,CU,1569514208,72,22.20,-84.09,79.69,8.32,base de Campismo LosPortales de Guane
7,Saldanha,0,PT,1569514226,27,41.42,-6.55,75.91,2.10,NaN
8,Shenjiamen,0,CN,1569514287,73,29.96,122.30,71.77,6.80,Jinjiang Inn Zhoushan Putuo Shenjiamen
9,Garlasco,0,IT,1569514304,60,45.20,8.92,77.00,3.36,I Diamanti Hotel


In [64]:
# * Plot markers on top of the heatmap.

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map